In [78]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

Setting up the webdriver
Plus, running the browser headless

In [79]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("/Users/asadimam/ChromeDriver/chromedriver-2",options=chrome_options)



In [83]:
# This is where we initialise all the necessary variables and lists    
# These are empty lists
size = [] 
desc = []
sale = []
location = []
property_type = []

# These are the necessary variables to run the loop
prop_type = {
            'Offices':1,
            'Industrial':2,
            'Land':3,
            'Retail':4,
            'Leisure':5,
            'Other':6
                }
prop_type_list = list(prop_type.values())
prop_type_list_string = list(map(str,prop_type_list))
locations = ['london','reading','maidenhead']

In [84]:
# This is the scraping code

for t, x in [(t,x) for t in prop_type_list_string for x in locations]:
        p= 1
        while True:
            url = 'https://www.novaloca.com/property-search-results/default.aspx?page={}&propertytypes={}&saletypes=1&status=1&sizefrom=1&sizeto=500000%2b&sizetype=2&location={}&distance=30&addresskeyword=&excludeServicedOffices=False'.format(p,t,x)
            driver.get(url)
            time.sleep(3)
            p += 1
            try:
                # Test to see if we're on the last page
                test = driver.find_element_by_class_name('extra-details')

#                 print(f'this is {p} of {x}')

                # Execution Code For Every Page
                ids = driver.find_elements_by_xpath("//*[contains(@id,'property-')]")
                property_ids = []
                for i in ids:
                    property_ids.append(i.get_attribute('id'))
                    for j in property_ids:
                        #     To extract sizes from each property-card
                        size_test = driver.find_elements_by_xpath('//*[@id="'+j+'"]/div/div[1]/div[3]/div/div[1]/div/span/strong')[0]    
                        size.append(size_test.text)
                        desc_test = driver.find_elements_by_xpath('//*[@id="'+j+'"]/div/div[2]/h4')[0]
                        desc.append(desc_test.text)
                        sale_test = driver.find_elements_by_xpath('//*[@id="'+j+'"]/div/div[2]/p/span[2]')[0]
                        sale.append(sale_test.text)
                        location.append(x)
                        property_type.append(t)

            except:
                break





In [85]:
df_properties_others = pd.DataFrame({'city':location,'prop_type':property_type,'prop_size':size,'prop_desc':desc,'prop_price':sale})

In [86]:
df_properties_others

,city,prop_type,prop_size,prop_desc,prop_price
0,london,1,"1,695 to 4,645 Sq Ft","Office / Residential Opportunity, 40 Great Smi...","- £3,400,000.00"
1,london,1,"1,695 to 4,645 Sq Ft","Office / Residential Opportunity, 40 Great Smi...","- £3,400,000.00"
2,london,1,"5,690 Sq Ft","B1 Unit or May Suit A1, A2, D1, D2 (STP), Pala...","- £2,250,000.00"
3,london,1,"1,695 to 4,645 Sq Ft","Office / Residential Opportunity, 40 Great Smi...","- £3,400,000.00"
4,london,1,"5,690 Sq Ft","B1 Unit or May Suit A1, A2, D1, D2 (STP), Pala...","- £2,250,000.00"
...,...,...,...,...,...
14348,maidenhead,6,"6,781 Sq Ft","Residential Development Opportunity, 165 & 167...","- £2,550,000.00"
14349,maidenhead,6,43579 to 305050 Sq Ft,"Viking Park, Petersfield Road, Kingsley, GU35 9AH","- £675,000.00"
14350,maidenhead,6,7408 Sq Ft,"Land Rear Of 73 Eden Road And 8 The Close, Ede...",
14351,maidenhead,6,261471 Sq Ft,"Development Site, Passfield Business Centre, L...",


In [ ]:
df_properties_others.to_csv("pro")